# Exploratory Data Analysis — Lahore AQI

This notebook analyzes AQI trends, pollutant distributions, and weather-AQI correlations
using backfilled data from the Feature Store.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.feature_pipeline.feature_store import get_features

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', 50)

print('Libraries loaded successfully!')

## 1. Load Data from Feature Store

In [ ]:
# Load features from Feature Store (local fallback if Hopsworks not configured)
df = get_features(use_hopsworks=False)

print(f'Dataset shape: {df.shape}')
print(f'Date range: {df["datetime"].min()} → {df["datetime"].max()}')
print(f'\nColumn types:\n{df.dtypes}')
df.head()

In [ ]:
# Basic statistics
df.describe().round(2)

In [ ]:
# Missing values
missing = df.isnull().sum()
missing = missing[missing > 0]
if missing.empty:
    print('No missing values!')
else:
    print('Missing values:')
    print(missing)

## 2. AQI Distribution & Trends

In [ ]:
# AQI distribution histogram
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['aqi'], bins=50, color='steelblue', edgecolor='white', alpha=0.8)
axes[0].set_xlabel('AQI Value')
axes[0].set_ylabel('Frequency')
axes[0].set_title('AQI Distribution — Lahore')
axes[0].axvline(df['aqi'].mean(), color='red', linestyle='--', label=f'Mean: {df["aqi"].mean():.0f}')
axes[0].legend()

# Box plot
axes[1].boxplot(df['aqi'], vert=True)
axes[1].set_ylabel('AQI Value')
axes[1].set_title('AQI Box Plot')

plt.tight_layout()
plt.show()

print(f'AQI Statistics:')
print(f'  Mean: {df["aqi"].mean():.1f}')
print(f'  Median: {df["aqi"].median():.1f}')
print(f'  Std: {df["aqi"].std():.1f}')
print(f'  Min: {df["aqi"].min():.0f}, Max: {df["aqi"].max():.0f}')

In [ ]:
# AQI time series
fig = px.line(df, x='datetime', y='aqi', title='AQI Over Time — Lahore',
              labels={'aqi': 'AQI', 'datetime': 'Date'})

# Add AQI category bands
fig.add_hrect(y0=0, y1=50, fillcolor='green', opacity=0.1, annotation_text='Good')
fig.add_hrect(y0=51, y1=100, fillcolor='yellow', opacity=0.1, annotation_text='Moderate')
fig.add_hrect(y0=101, y1=150, fillcolor='orange', opacity=0.1, annotation_text='Unhealthy (SG)')
fig.add_hrect(y0=151, y1=200, fillcolor='red', opacity=0.1, annotation_text='Unhealthy')
fig.add_hrect(y0=201, y1=300, fillcolor='purple', opacity=0.1, annotation_text='Very Unhealthy')

fig.update_layout(height=500)
fig.show()

## 3. Temporal Patterns

In [ ]:
# Hourly pattern
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# By hour
hourly = df.groupby('hour')['aqi'].mean()
axes[0].bar(hourly.index, hourly.values, color='steelblue')
axes[0].set_xlabel('Hour of Day')
axes[0].set_ylabel('Average AQI')
axes[0].set_title('Average AQI by Hour')

# By day of week
daily = df.groupby('day_of_week')['aqi'].mean()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1].bar(range(7), daily.values, color='coral', tick_label=days)
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Average AQI')
axes[1].set_title('Average AQI by Day of Week')

# By month
if 'month' in df.columns:
    monthly = df.groupby('month')['aqi'].mean()
    axes[2].bar(monthly.index, monthly.values, color='seagreen')
    axes[2].set_xlabel('Month')
    axes[2].set_ylabel('Average AQI')
    axes[2].set_title('Average AQI by Month')

plt.tight_layout()
plt.show()

In [ ]:
# Weekend vs Weekday comparison
weekend_aqi = df[df['is_weekend'] == 1]['aqi'].mean()
weekday_aqi = df[df['is_weekend'] == 0]['aqi'].mean()
print(f'Weekday average AQI: {weekday_aqi:.1f}')
print(f'Weekend average AQI: {weekend_aqi:.1f}')
print(f'Difference: {weekday_aqi - weekend_aqi:.1f}')

## 4. Pollutant Analysis

In [ ]:
# Pollutant distributions
pollutants = ['pm2_5', 'pm10', 'no2', 'so2', 'co', 'o3']
available_pollutants = [p for p in pollutants if p in df.columns]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, pol in enumerate(available_pollutants):
    axes[i].hist(df[pol], bins=40, color=sns.color_palette('husl', 6)[i], edgecolor='white', alpha=0.8)
    axes[i].set_title(f'{pol.upper()} Distribution')
    axes[i].set_xlabel('Concentration (µg/m³)')
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Pollutant time series (interactive)
fig = make_subplots(rows=3, cols=2, subplot_titles=[p.upper() for p in available_pollutants])

for i, pol in enumerate(available_pollutants):
    row = i // 2 + 1
    col = i % 2 + 1
    fig.add_trace(go.Scatter(x=df['datetime'], y=df[pol], mode='lines', name=pol.upper()),
                  row=row, col=col)

fig.update_layout(height=800, title_text='Pollutant Concentrations Over Time')
fig.show()

## 5. Correlation Analysis

In [ ]:
# Correlation heatmap
corr_cols = available_pollutants + ['aqi', 'temperature', 'humidity', 'wind_speed', 'pressure']
corr_cols = [c for c in corr_cols if c in df.columns]

corr_matrix = df[corr_cols].corr()

fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, linewidths=0.5, ax=ax)
ax.set_title('Feature Correlation Matrix', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Top correlations with AQI
if 'aqi' in df.columns:
    aqi_corr = corr_matrix['aqi'].drop('aqi').sort_values(ascending=False)
    print('\nCorrelation with AQI (sorted):')
    for feat, corr_val in aqi_corr.items():
        bar = '█' * int(abs(corr_val) * 20)
        sign = '+' if corr_val > 0 else '-'
        print(f'  {feat:20s} {sign}{abs(corr_val):.3f} {bar}')

## 6. Time Series Decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Resample to daily for decomposition
daily_aqi = df.set_index('datetime')['aqi'].resample('D').mean().dropna()

if len(daily_aqi) >= 14:  # Need at least 2 weeks for decomposition
    decomposition = seasonal_decompose(daily_aqi, model='additive', period=7)

    fig, axes = plt.subplots(4, 1, figsize=(14, 12))
    decomposition.observed.plot(ax=axes[0], title='Observed')
    decomposition.trend.plot(ax=axes[1], title='Trend')
    decomposition.seasonal.plot(ax=axes[2], title='Seasonal (7-day)')
    decomposition.resid.plot(ax=axes[3], title='Residual')
    plt.tight_layout()
    plt.show()
else:
    print(f'Not enough data for decomposition ({len(daily_aqi)} days, need ≥14)')

## 7. Outlier Detection

In [ ]:
# IQR-based outlier detection for AQI
Q1 = df['aqi'].quantile(0.25)
Q3 = df['aqi'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = df[(df['aqi'] < lower) | (df['aqi'] > upper)]
print(f'AQI Outlier Detection (IQR method):')
print(f'  Q1: {Q1:.0f}, Q3: {Q3:.0f}, IQR: {IQR:.0f}')
print(f'  Lower bound: {lower:.0f}, Upper bound: {upper:.0f}')
print(f'  Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}% of data)')

if not outliers.empty:
    fig = px.scatter(df, x='datetime', y='aqi', title='AQI with Outliers Highlighted',
                     color=(df['aqi'] > upper) | (df['aqi'] < lower),
                     color_discrete_map={True: 'red', False: 'steelblue'})
    fig.show()

## 8. Key Insights Summary

Run all cells above, then summarize:
- AQI distribution shape (normal, skewed, bimodal?)
- Peak pollution hours and days
- Seasonal patterns (winter vs summer in Lahore)
- Most correlated features with AQI
- Data quality observations